<a href="https://colab.research.google.com/github/LillySh/WishList/blob/main/NLP_Potapova_HW3_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Метод similarity

У всех типов объектов-контейнеров в библиотеке spaCy есть метод similarity, который используется для вычисления оценки семантического подобия двух объектов-контейнеров произвольного типа посредством сравнения их векторов слов. Для вычисления подобия интервалов и документов, не обладающих векторами слов, spaCy сравнивает средние значения векторов слов содержащихся в них
токенов.

Вычислим семантическое подобие двух объектов-контейнеров для различных объектов. В коде ниже вычислим оценку семантического подобия между предложением I want a green apple. и вырезанной из него фразой a green apple.

Как видим, вычисленная оценка подобия достаточно высока, чтобы считать содержимое обоих объектов схожим (диапазон значений степени подобия от 0 до 1).

In [1]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc=nlp('I want a green apple.')
doc.similarity(doc[2:5])

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
<ipython-input-1-94f975ecb8dd>:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  doc.similarity(doc[2:5])


0.6240444557468489

Сравнение объекта самого с собой. Логично, что вернется значение 1.

In [2]:
doc.similarity(doc)

1.0

In [3]:
doc[2:5].similarity(doc[2:5])

1.0

Далее сравним хранящееся в doc2 предложение I like red oranges и интервал a green apple, извлеченный из doc. 

В данном случае степень подобия оказалась мала.  Так как указаны разные фрукты в предложении (метод similarity это учитывает) и глаголы want и like выражают разные состояния.

In [4]:
doc2=nlp('I like red oranges.')
doc2.similarity(doc[2:5])

<ipython-input-4-29dc56b362d7>:2: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  doc2.similarity(doc[2:5])


0.22478505229070148

Далее сравним объект Token oranges с объектом Span, содержащим один токен apple.

In [6]:
token = doc2[3:4][0]
token

oranges

In [7]:
token.similarity(doc[4:5])

<ipython-input-7-2c624528aea6>:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  token.similarity(doc[4:5])


0.5848100781440735

Прежде всего, с помощью ссылки на первый элемент интервала преобразовали явным образом объект Span с одним токеном oranges в объект Token, а затем вычислили степень его подобия интервалу apple.

Метод similarity() распознает слова, которые относятся к одной или
нескольким близким категориям и часто встречаются во взаимосвязанных контекстах, а также возвращает для подобных слов высокие
значения уровня подобия.

### Применение семантического подобия для задач категоризации

Вычисление семантического подобия двух объектов позволяет распределять тексты по категориям, а также выбирать из корпуса лишь те из них, которые относятся к нужной теме. 

Далее из всех комментариев пользователей на сайте нам нужно выбрать комментарии, связанные со словом fruits. В нашей задаче мы будем анализировать следующие комментарии:
* I want to buy this beautiful book at the end of the week.
* Sales of citrus have increased over the last year.
* How much do you know about this type of tree?


Чтобы выбрать это с помощью кода, необходимо сравним каждое предложение со словом fruits. 

Далее создадим объект Token для слова fruits и применим конвейер к категоризируемым предложениям, создавая для них общий объект Doc. Далее разрезаем его на предложения, а затем выводим в консоль каждое из них и его семантическое подобие к токену fruits, которое вычисляется с помощью метода similarity объекта этого токена.

In [8]:
import spacy
nlp = spacy.load('en_core_web_sm')
token = nlp(u'fruits')[0]
doc = nlp(u'I want to buy this beautiful book at the end of the week. Sales of citrus have increased over the last year. How much do you know about this type of tree?')
for sent in doc.sents: #разделение на предложения
  print(sent.text)
  print('similarity to', token.text, 'is', token.similarity(sent),'\n')

I want to buy this beautiful book at the end of the week.
similarity to fruits is -0.04223029315471649 

Sales of citrus have increased over the last year.
similarity to fruits is -0.027368908748030663 

How much do you know about this type of tree?
similarity to fruits is -0.07132802903652191 



<ipython-input-8-52ad01cea877>:7: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print('similarity to', token.text, 'is', token.similarity(sent),'\n')


Видим, что степень подобия слова fruits к каждому предложению крайне мала. Это еще зависит от используемой модели.

### Выделение существительных как шаг предварительной обработки

Более удачный подход к категоризации -  выделять только самые значимые слова и сравнивать только их. Подобная подготовка текста к обработке, которая называется предварительной обработкой, позволяет существенно повысить успешность опе-
раций NLP.

То есть теперь вместо сравнения векторов слов для целого объекта будем сравнивать векторы слов только для определенных частей речи.

Начнем с описания токена fruits, который будем использовать для ряда
сравнений. Проходя в цикле по токенам каждого предложения, выделяем существительные и сохраняем их в списке Python. Затем склеиваем существительные из этого списка в простую строку,
которую позже преобразуем в объект Doc. Далее сравниваем объект Doc с токеном fruits, чтобы определить степень их семантического подобия. Значение семантического подобия сохраняется в ассоциативном
массиве Python, который затем выводится в консоль

In [9]:
import spacy
nlp = spacy.load('en_core_web_sm')
token = nlp(u'fruits')[0] #описания токена
doc = nlp(u'I want to buy this beautiful book at the end of the week. Sales of citrus have increased over the last year. How much do you know about this type of tree?')
similarity = {}
for i, sent in enumerate(doc.sents):#проходим по каждому предложению
  noun_span_list = [sent[j].text for j in range(len(sent)) if #выделяем существительные и записываем их
  sent[j].pos_ == 'NOUN']
  noun_span_str = ' '.join(noun_span_list) #склеиваем существительные в строку
  noun_span_doc = nlp(noun_span_str) #преобразуем строку с существительными в объект doc
  similarity.update({i:token.similarity(noun_span_doc)}) #подобие сохраняем в ассоциативный массив
print(similarity)

{0: 0.0726100970506393, 1: 0.2306078439472607, 2: 0.31927317721232235}


<ipython-input-9-8b19815d5d94>:11: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity.update({i:token.similarity(noun_span_doc)}) #подобие сохраняем в ассоциативный массив


Как видим степень семантического подобия со словом fruits выше для всех предложений.

## Выделение и сравнение именованных сущностей


В некоторых случаях вместо выделения всех существительных из сравниваемых текстов имеет смысл выделять лишь определенный вид,например именованные сущности.

Далее в коде мы группируем объекты Doc с примерами текстов в список, чтобы по ним можно было пройти в цикле. Объявляем ассоциативный массив Python для хранения ключевых слов всех текстов. Проходим в цикле по объектам Doc, выделяя ключевые слова в отдельные списки для каждого из текстов. При этом выбор падает лишь на слова, маркированные как именованные сущности. Далее выводим список в консоль, чтобы посмотреть его содержимое. Трансформируем этот список в простую строку, к которой применяем конвейер, а затем преобразуем ее в объект Doc. Наконец, добавляем объект Doc к объявленному ранее ассоциативному массиву spans.

In [10]:
import spacy
nlp = spacy.load('en_core_web_sm')

doc1 = nlp(u'Google Search, often referred to as simply Google, is the most used search engine nowadays. It handles a huge number of searches each day.')

doc2 = nlp(u'Microsoft Windows is a family of proprietary operating systems developed and sold by Microsoft. The company also produces a wide range of other software for desktops and servers.')

doc3 = nlp(u"Titicaca is a large, deep, mountain lake in the Andes. It is known as the highest navigable lake in the world.")
docs = [doc1,doc2,doc3] #группировка объектов
spans = {}#объявление ассоциативного массива для хранения ключевых слов
for j,doc in enumerate(docs):#проход в цикле по объектам, выделяя ключевые слова
  named_entity_span = [doc[i].text for i in range(len(doc)) if#ключевые являются существительные
  doc[i].ent_type != 0]
  print(named_entity_span)#вывод в консоли
  named_entity_span = ' '.join(named_entity_span)#трансформация в простую строку, которая преобразуется в конвейер
  named_entity_span = nlp(named_entity_span)#преобразование в объект doc
  spans.update({j:named_entity_span})#добавление объект doc к массиву

['Google', 'Search', 'Google', 'each', 'day']
['Microsoft', 'Windows', 'Microsoft']
['Titicaca', 'Andes']


Таким образом, мы получили набор именованных сущностей из каждогт предложения.

Теперь вызываем метод similarity() для каждого из интервалов и выводим результаты


In [11]:
print('doc1 is similar to doc2:',spans[0].similarity(spans[1]))
print('doc1 is similar to doc3:',spans[0].similarity(spans[2]))
print('doc2 is similar to doc3:',spans[1].similarity(spans[2]))

doc1 is similar to doc2: 0.7054618244550298
doc1 is similar to doc3: 0.6104506459201123
doc2 is similar to doc3: 0.6308411086910213


<ipython-input-11-3ba975a8ccfc>:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print('doc1 is similar to doc2:',spans[0].similarity(spans[1]))
<ipython-input-11-3ba975a8ccfc>:2: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, 

Таким образом, полученные значения указывают, что наиболее подобны первый и второй тексты, посвященные американским IT-компаниям.

## Поиск паттернов лингвистических признаков

Поиск паттернов в текстах необходимо по той причине, что в большинстве случаев в тексте не бывает двух совершенно одинаковых предложений, так как обычно текст состоит из разных предложений, содержащих разные слова. Так как писать отдельный код для обработки каждого предложения текста нерационально.

Поскольку число слов в анализируемых предложениях одинаково, можно пройти по словам обоих предложений в одном цикле. Если метка зависимости одинакова для слов с идентичными индексами, то выводим эти слова, присвоенную им метку и ее описание.

In [12]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc1 = nlp(u'We can overtake them.')
doc2 = nlp(u'You must specify it.')
for i in range(len(doc1)-1):
  if doc1[i].dep_ == doc2[i].dep_:
    print(doc1[i].text, doc2[i].text, doc1[i].dep_,
  spacy.explain(doc1[i].dep_))

We You nsubj nominal subject
can must aux auxiliary
overtake specify ROOT root
them it dobj direct object


Видим, что список меток зависимостей для обоих предложений. Это означает, что предложение строятся по одному паттерну последовательности слов, который описывается метками синтакси-
ческой зависимости таким образом: «подлежащее + вспомогательный глагол + глагол + прямое дополнение».

## Проверка высказывания на соотвествие паттерну.

Опишем функцию, реализующую указанный паттерн, а затем протестируем ее на примере предложения:

Ниже описана функция dep_pattern, принимающая в качестве- параметра объект Doc. В ней проходим в цикле по токенам объекта Doc и ищем паттерн «подлежащее + вспомогательный глагол + глагол».


Найдя его, проверяем, есть ли среди непосредственных дочерних элементов глагола прямое дополнение. В случае обнаружения этого прямого дополнения функция вернет True, в противном случае — False

In [13]:
import spacy
nlp = spacy.load('en_core_web_sm')
def dep_pattern(doc):#принимаем параметр объекта doc
  for i in range(len(doc)-1):#проход в цикле
    if doc[i].dep_ == 'nsubj' and doc[i+1].dep_ == 'aux' and doc[i+2].dep_ == 'ROOT':#ищем паттерн подлежащее+вспом глагол+глагол
      for tok in doc[i+2].children:#проверка на наличие прямого дополнения
        if tok.dep_ == 'dobj':
          return True#нашли прямое дополнение
  return False#
doc = nlp(u'We can overtake them.')#передаем объект doc
if dep_pattern(doc):#
  print('Found')
else:
  print('Not found')

Found


Так как образец предложения из данного примера соответствует
указанному паттерну, сценарий выдаст в консоль:

`Found`

## Использование утилиты Matcher библиотеки spaCy для поиска паттернов последовательностей слов.


В библиотеке spaCy есть встроенная утилита для решения подобной задачи — Matcher,
специально разработанная для поиска последовательностей токенов, соответствующих заданному паттерну.

Ниже реализуем паттерн «подлежащее + вспомогательный глагол + глагол» с помощью Matcher 

Создаем экземпляр Matcher, передавая конструктору объект со словарным запасом, соответствующим документам, которые этот Matcher будет обрабатывать. Далее описываем паттерн, указывая метки зависимости, которым должна удовлетворять последовательность слов.
Добавляем только что созданный паттерн в Matcher.
Применяем Matcher к нашему примеру текста и получаем список соответствующих паттерну токенов, а затем проходим по этому списку
в цикле, выводя начальную и конечную позиции токенов паттерна в тексте

In [ ]:
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)#создаем экземпляр
pattern = [{"DEP": "nsubj"}, {"DEP": "aux"}, {"DEP": "ROOT"}]#описание паттерна
matcher.add("NsubjAuxRoot", [pattern])#применяем matcher к тексту и получаем список соответсвующих токенов
doc = nlp(u"We can overtake them.")
matches = matcher(doc)#
for match_id, start, end in matches:#проход в цикле по списку
  span = doc[start:end]
  print("Span: ", span.text)#выводим начальную и конечную позицию токенов
  print("The positions in the doc are: ", start, "-", end)

Span:  We can overtake
The positions in the doc are:  0 - 3


В результате получаем начальную и конечную позицию слов, составляющих последовательность, которая
соответствует заданному паттерну.

Такой подход очень удобен для последовательностей слов, которые расположены одно за другим.

## Применение нескольких паттернов

Помимо последовательности меток зависимостей, описанной в подразделе «Проверка высказывания на соответствие паттерну» можно описать новую функцию, реализующую паттерн, основанный на
тегах частей речи. Например, описываемый тегами частей речи паттерн будет содержать условие, что подлежащее и прямое дополнение должны быть личными местоимениями. Эта новая функция может среди про-
чего реализовывать паттерн «личное местоимение + вспомогательный модальный глагол + глагол в инфинитиве + ... + личное местоимение...».

Функция описанная ниже в случае обнаружении несоответствия возвращает False. В противном случае по завершении всех проверок и при отсутствии каких-либо несоответствий функция возвращает True.

Для проверки на паттерны применяем к предложению конвейер, после чего проверяем, удовлетворяет ли оно обоим паттернам. Поскольку используемый в этом примере образец предложения удовлетворяет обоим паттернам, должно появиться следующее:

`Found`

Если же заменить образец предложения на такой: I might send them a card as a reminder., будет выведено:

`Not found`



In [14]:
import spacy
nlp = spacy.load('en_core_web_sm')

def pos_pattern (doc):#описываем функцию
  for token in doc:
    #Проверка на соответствие части предложения определенному тегу
    if token.dep_ == 'nsubj' and token.tag_ != 'PRP':
      return False
    if token.dep_ == 'aux' and token.tag_ != 'MD':
      return False
    if token.dep_ == 'ROOT' and token.tag_ != 'VB':
      return False
    if token.dep_ == 'dobj' and token.tag_ != 'PRP':
      return False#при несоответствии 
  return True#соответствие
#Тестирование кода
doc = nlp(u'We can overtake them.')
if dep_pattern(doc) and pos_pattern(doc):#проверка удовлетворению
  print('Found')
else:
  print('Not found')

Found


Таким образом, полученный результат означает, что используемый в этом примере образец предложения удовлетворяет обоим паттернам.

### Создание паттернов на основе пользовательских признаков

При создании паттерна последовательности слов может возникнуть необходимость расширить функциональность лингвистических признаков, предоставляемых spaCy, подогнав их под свои задачи. Например, может понадобиться, чтобы предыдущий сценарий распознавал паттерн, в котором учитывается число местоимений (единственное или множественное). Это полезно в том случае, если нужно найти существительное, к которому относится местоимение. 

Однако SpaCy не различает число местоимений.

В функции pron_
pattern описываем список Python, включающий все возможные личные местоимения во множественном числе. Далее описываем цикл,
в котором проходим по токенам полученного предложения в поисках
прямого дополнения — личного местоимения. Найдя, проверяем,
встречается ли оно в списке личных местоимений во множественном
числе. Если да, наша функция возвращает plural. В противном
случае она возвращает singular. Если функции не удалось обнаружить прямое дополнение или оно не является личным местоимением,
возвращается Not found.

In [ ]:
def pron_pattern(doc):
  plural = ['we','us','they','them']#описываем список, включающий личные местоимения
  for token in doc:
    if token.dep_ == 'dobj' and token.tag_ == 'PRP':#поиск прямого дополнения - личного местоимения
      if token.text in plural:
        return 'plural'#при нахождении
      else:
        return 'singular'#при отсутствии
  return 'not found'#если нет прямого дополнения или оно не является личным местоимением
doc = nlp(u'We can overtake them.')
if dep_pattern(doc) and pos_pattern(doc):
  print('Found:', 'the pronoun in position of direct object is',
  pron_pattern(doc))
else:
  print('Not found')

Found: the pronoun in position of direct object is plural


Как видим, функция вернула `plural`, что означает, что в предложении встретилось личное местоимение во множественном числе.

## Применение паттернов последовательностей слов в чат-бот для генерации высказываний

Как упоминалось ранее, самые трудные задачи NLP — понимание и генерация текста на естественных языках. Чат-бот должен понимать вводимый пользователем текст и генерировать на него должный ответ.
Паттерны последовательностей слов, основанные на лингвистических признаках, могут быть полезны при реализации этих функций.

С помощью паттернов последовательностей слов можно реагировать на
высказывания пользователя и другим образом — например, генерируя
подходящие утвердительные высказывания.

функция find_noun, принимающая два параметра.
Первый из них содержит список предложений от начала текста до
предложения, удовлетворяющего всем паттернам. В данном примере этот список будет включать все предложения текста, поскольку
только последнее из них удовлетворяет всем паттернам. Однако
существительное, обозначаемое местоимением, находится в одном из
предыдущих предложений.
Второй из передаваемых функции find_noun параметров — число местоимения, выступающего в роли прямого дополнения, которое удовлетворяет всем паттернам. Его определяет функция pron_pattern.
Если значение этого аргумента 'plural', описываем список Python,
содержащий теги уточненных частей речи, которые в spaCy отмечают
существительные во множественном числе. Если же 'singular',
создаем список тегов общих частей речи, обозначающих существительные в единственном числе.

In [ ]:
def find_noun(sents, num):#первый параметр содержит список предложений от начала текста до предложения, удовлет всем паттернам. 
#второй параметр - число местоимений, являющ прямым дополнением, удовлет всем паттернам
  if num == 'plural':
    taglist = ['NNS','NNPS']#для сущ во множ числе
  if num == 'singular':
    taglist = ['NN','NNP']#для сущ в единственном числе
  for sent in reversed(sents):#проход в обратном порядке, с ближайшего предложения
    for token in sent:#проход по токенам в каждом предложении
      if token.tag_ in taglist:#происк токена, тег уточнения части речи
        return token.text
  return 'Noun not found'

Проходим в цикле for по токенам предложения и ищем личное местоимение, которое выступает в роли прямого дополнения. Найдя его, генерируем новое высказывание. Меняем исходное предложение, заменяя личное местоимение соответствующим существительным и добавляя в конец too. После этого функция gen_utterance возвращает только что сгенерированное утвердительное высказывание.
Если же найти прямое дополнение в форме личного местоимения не удалось, функция возвращает сообщение об ошибке 

In [ ]:
def gen_utterance(doc, noun):#функция генерации нового утвердительного высказывания
  sent = ''
  for i,token in enumerate(doc):#проход по токенам
    if token.dep_ == 'dobj' and token.tag_ == 'PRP':#поиск личных местоимений, являющ дополнением
      sent = doc[:i].text + ' ' + noun + ' ' + doc[i+1:len(doc)-2].text + 'too.'#меняем личные местоимения на сущ
      return sent#возвращение, что генерация прошла
  return 'Failed to generate an utterance'#в случае если не было найдено

После применения конвейера к нашему образцу текста преобразуем текст в список предложений. Проходим в цикле по этому
списку и ищем предложение, которое удовлетворяет описанным
в функциях dep_pattern и pos_pattern паттернам. Затем с помощью
функции find_noun определяем существительное, соответствующее
местоимению из предложения, найденного на предыдущем шаге.
Наконец, вызываем функцию gen_utterance для генерации соответствующего высказывания

In [ ]:
#проверка работы
doc = nlp(u'The symbols are clearly distinguishable. I can recognize them promptly.')#заведем текст
sents = list(doc.sents)#преобразование текста в список предложений
response = ''
noun = ''
for i, sent in enumerate(sents):#проходим по циклу, ищем удовлетворяющих предложений
  if dep_pattern(sent) and pos_pattern(sent):
    noun = find_noun(sents[:i], pron_pattern(sent))#определяем сущ, соответствующее местоим из предложения
    if noun != 'Noun not found':
      response = gen_utterance(sents[i],noun)#вызываем функцию для генерации высказывания
    break
print(response)

I can recognize symbols too.


## Проход в цикле по главным элементам токенов

Теперь разберемся, как выразить отношение между словами to и Berlin
программным образом. Один из способов — обход дерева зависимостей
слева направо, начиная с to, с проходом только по правосторонним
дочерним элементам всех токенов. Если подобным образом можно
пройти от to до Berlin, значит, можно обоснованно считать, что между
этими двумя словами существует семантическая связь.

В функции det_destination проходим в цикле по токенам полученного
предложения в поисках сущности типа GPE. В случае ее обнаружения
начинаем цикл while, в котором проходим по главным элементам всех
токенов, начиная с токена, содержащего сущность типа GPE. Цикл завершается по достижении либо токена, содержащего to, либо корня предложения. Найти корень предложения можно путем сравнения каждого
токена с его главным элементом, поскольку главный элемент корневого
токена всегда ссылается на него. Или же можно проверять тег ROOT

In [ ]:
def det_destination(doc):#функция для определения места назначения
  for i, token in enumerate(doc):
    if token.ent_type != 0 and token.ent_type_ == 'GPE':#проход по токенам в поиске сущности типа GPE
      while True:#в случае, если найдено
        token = token.head#проход по главным элементам токенов
        if token.text == 'to':
          return doc[i].text#завершение текста до to
        if token.head == token:#сравнение токена с главным элементом
          return 'Failed to determine'
  return 'Failed to determine'

doc = nlp(u'I am going to the conference in Berlin.')
dest = det_destination(doc)#
print('It seems the user wants a ticket to ' + dest)

It seems the user wants a ticket to Berlin


### Краткое изложение текста с помощью деревьев зависимостей

Конечно, сфера применения подхода с деревьями синтаксических
зависимостей не ограничивается чат-ботами. Этот подход можно

использовать, например, в приложениях, предназначенных для обра-
ботки отчетов. Представьте, что вам нужно разработать приложение

для краткого изложения отчетов о розничных продажах, чтобы в них
была лишь самая важная информация.

Например, можно извлечь только предложения, содержащие числа. Та-
ким образом получится краткий отчет об объемах продаж, доходах и за-
тратах.

Проходим в цикле по токенам предложения в поисках токена, представляющего число. Когда таковой обнаруживаем, начинаем выполнение
цикла while, в котором проходим по правосторонним главным элементам, стартуя с токена числа. Далее формируем новую фразу, присоединяя текст всех главных элементов в конец переменной phrase. С целью убедиться, что главный элемент следующего токена располагается
справа от него, проверяем, включен ли токен в список левосторонних
дочерних элементов его главного элемента.

In [ ]:
doc = nlp(u"The product sales hit a new record in the first quarter, with 18.6 million units sold.")
phrase = ''
for token in doc:
  if token.pos_ == 'NUM':#поиск токена - число
    while True:
      #проходим по правосторонним главным элементам, начиная с токена числа
      phrase = phrase + ' ' + token.text
      token = token.head#формируем новую фразу, присоединяя в конец
      if token not in list(token.head.lefts):#проверка включен ли токен в список левосторонних дочерних элементов его главного элемента
        phrase = phrase + ' ' + token.text
      if list(token.rights):
        phrase = phrase + ' ' + doc[token.i+1:].text
        break#прерывание, если условие ложно
    break#прерывание внешнего
print(phrase.strip())

18.6 million units sold.


Обходим главные элементы токенов в цикле while, добавляя в конец
формируемой фразы текст из этих элементов. По достижении смыслового глагола (с меткой ROOT)  прерываем выполнение цикла.

In [ ]:
while True:
  token = doc[token.i].head#обход главного элемента
  if token.pos_ != 'ADP':
    phrase = token.text + phrase#добавление в конец элемент
  if token.dep_ == 'ROOT':#при достижении глагола 
    break#завершение

Начинаем с обхода в цикле дочерних элементов смыслового глагола в поисках подлежащего. Затем присоединяем к началу дочерние элементы подлежащего и подлежащее фразы, после чего выводим полученную фразу.

In [ ]:
for tok in token.lefts:#обход в цикле дочернего элемента смыслового глагола
  if tok.dep_ == 'nsubj':#поиск подлежащего
    phrase = ' '.join([tok.text for tok in tok.lefts]) + ' ' + tok.text + ' ' + phrase
    #присоединение к началу дочерние элементы подлежащего и подлежащее фразы
    break
print(phrase)#получаем фразу

The product sales hit 18.6 million units sold.


Усовершенствование чат-бота для бронирования билетов с помощью учета контекста

Код в функции guess_destination проходит в цикле по токенам предложения в поисках сущности типа GPE . После обнаружения функция
возвращает ее вызывающему коду . Если же найти такую сущность
не удалось, функция возвращает сообщение 'Failed to determine' ,
означающее, что предложение не содержит сущности типа GPE.

In [ ]:
#функция поиска в предложении GPE
def guess_destination(doc):
  for token in doc:
    if token.ent_type != 0 and token.ent_type_ == 'GPE':#проход по циклу в поиске GPE
      return token.text#в случае нахождения возвращает
  return 'Failed to determine'#если не удалось найти

Код функции gen_response начинается с вызова функции det_destination, которая определяет, содержит ли данное высказывание пару to + GPE. В случае обнаружения такой пары считаем, что пользователь хочет забронировать билет в указанный пункт назначения,
поэтому уточняем время вылета.


In [ ]:
#Генерация ответа
def gen_response(doc):
  dest = det_destination(doc)#вызов функции места назначения
  if dest != 'Failed to determine':
    return 'When do you need to be in ' + dest + '?'#уточняем время вылета в место назначения
  dest = guess_destination(doc)#функция для поиска в предложении GPE
  if dest != 'Failed to determine':
    return 'You want a ticket to ' + dest +', right?'#уточнение действительно ли необходим вылет
  return 'Are you flying somewhere?'#

Для проверки кода в деле применяем к предложению конвейер, а затем
передаем объект Doc в функцию gen_response, описанную в предыдущем листинге:


In [ ]:
#проверка работы функции
doc = nlp(u'I am going to the conference in Berlin.')
print(gen_response(doc))

When do you need to be in Berlin?


Повышение IQ чат-бота за счет поиска подходящих модификаций

Модификатор (modifier) — необязательный элемент фразы или простого предложения, меняющий смысл другого элемента. Удаление модификатора не всегда меняет основной смысл предложения, но делает его более общим.

Начинаем с применения конвейера к короткому тексту, содержащему слово fruit с пре- и постмодификаторами. Объявляем два пустых списка: fruit_adjectives  и fruit_origins. В первом будут храниться все
найденные для слова fruit прилагательные-модификаторы. Во втором —все найденные среди постмодификаторов слова fruit сущности типа GPE.

Далее ищем в цикле по токенам всего текста слово fruit. Найдя его, сначала определяем прилагательные-премодификаторы этого слова,
проходя по его левосторонним дочерним элементам и выбирая только прилагательные (определители и составные слова также могут быть
премодификаторами). Добавляем прилагательные-модификаторы
в список fruit_adjectives.

Затем ищем постмодификаторы — в частности, именованные сущности, — проходя по правосторонним синтаксическим дочерним элементам слова fruit, и добавляем их в список fruit_origins.

Сценарий выводит два списка:

The list of adjectival modifiers for word fruit: ['nice', 'exotic']

The list of GPE names applicable to word fruit as postmodifiers: ['Africa']

Теперь наш бот «знает», что фрукт может быть приятным на вкус, экзотическим (или же и приятным, и экзотическим одновременно), а также может поставляться из Африки.

In [ ]:
doc = nlp(u"Kiwano has jelly-like flesh with a refreshingly fruity taste. This is a nice exotic fruit from Africa. It is definitely worth trying.")
fruit_adjectives = []#объявление списка, будут храниться все найденные для слова fruit прилагательные-модификаторы.
fruit_origins = []#объявление списка все найденные среди постмодификаторов слова fruit сущности типа GPE.
for token in doc:
  if token.text == 'fruit':#поиск слова fruit по токенам
  #определим прилагательные-премодификаторы слова, проходя по левосторонним элементам и выбирая прилагательные
  #добавляем прилагательные в 1 список
    fruit_adjectives = fruit_adjectives + [modifier.text for modifier
    in token.lefts if modifier.pos_ == 'ADJ']
  #ищем постмодификаторы - именованные сущности по правосторонним синтаксическим доч. элементам и добавляем во второй список
    fruit_origins = fruit_origins + [doc[modifier.i + 1].text for modifier
    in token.rights if modifier.text == 'from' and
doc[modifier.i + 1].ent_type != 0]
#вывод вкуса
print('The list of adjectival modifiers for word fruit:',
  fruit_adjectives)
#вывод места поставки
print('The list of GPE names applicable to word fruit as postmodifiers:',
  fruit_origins)

The list of adjectival modifiers for word fruit: ['nice', 'exotic']
The list of GPE names applicable to word fruit as postmodifiers: ['Africa']
